# Notebook to generate indicators module

In [120]:

from pprint import pformat

from inspect import Signature, Parameter

from mintalib import core
from mintalib.samples import sample_prices

from __project__ import PKGDIR

In [121]:
prices = sample_prices()
prices.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11033 entries, 1980-12-12 to 2024-09-18
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    11033 non-null  float64
 1   high    11033 non-null  float64
 2   low     11033 non-null  float64
 3   close   11033 non-null  float64
 4   volume  11033 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 517.2 KB


In [122]:


import importlib.util

def new_module(name: str, code: str = None):
    spec = importlib.util.spec_from_loader(name, None)
    module = importlib.util.module_from_spec(spec)
    if code:
        exec(code, module.__dict__)
    return module

In [123]:
def core_functions():
    return tuple(k for k, v in vars(core).items() if k.isupper() and callable(v))

core_functions()

('AVGPRICE',
 'TYPPRICE',
 'WCLPRICE',
 'MIDPRICE',
 'PRICE',
 'CROSSOVER',
 'CROSSUNDER',
 'FLAG_ABOVE',
 'FLAG_BELOW',
 'FLAG_INVERT',
 'FLAG_UPDOWN',
 'LOG',
 'EXP',
 'ROC',
 'DIFF',
 'MIN',
 'MAX',
 'SUM',
 'MAD',
 'STDEV',
 'SMA',
 'EMA',
 'RMA',
 'WMA',
 'HMA',
 'DEMA',
 'TEMA',
 'MA',
 'RSI',
 'PLUSDI',
 'MINUSDI',
 'ADX',
 'TRANGE',
 'ATR',
 'NATR',
 'LATR',
 'SAR',
 'CCI',
 'CMF',
 'MFI',
 'BOP',
 'BBANDS',
 'KELTNER',
 'KER',
 'KAMA',
 'MACD',
 'PPO',
 'SLOPE',
 'RVALUE',
 'FORECAST',
 'STOCH',
 'STREAK',
 'EVAL')

In [124]:
# Sample code for indicator

from mintalib.model import Indicator
from mintalib.core import wrap_indicator

@wrap_indicator(core.EMA)
class EMA(Indicator):
    same_scale = True

    def __init__(self, period: int, *, item: str = None):
        self.period = period
        self.item = item

    def __call__(self, series):
        return core.EMA(series, period=self.period)

In [125]:
PREAMBLE = '''
""" Mintalib Indicators """

# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from . import model

nan = float('nan')

'''


def make_indicator(func, name: str = None, preamble=False, verbose=False):
    if name is None:
        name = func.__name__.upper()

    fname = func.__qualname__
    module = func.__module__.rpartition(".")[2]
    qname = f"{module}.{fname}"

    indent = " " * 4
    indent2 = " " * 8

    signature = Signature.from_callable(func)
    parameters = list(signature.parameters.values())

    if verbose:
        print("signature", signature)
        print("parameters", parameters)

    newparams = []
    ftype = parameters[0].name

    def annotate(param):
        if param.name in ["period"]:
            return param.replace(annotation=int)

        if param.annotation == "str":
            return param.replace(annotation=str)

        if type(p.default) in (int, float, bool):
            return param.replace(annotation=type(p.default))

        return param

    for i, p in enumerate(parameters):
        if i == 0:
            p = Parameter("self", Parameter.POSITIONAL_OR_KEYWORD)
        else:
            p = annotate(p)
        newparams.append(p)

    if verbose:
        print("newparams", newparams)

    newsig = Signature(newparams)

    def argument(arg):
        return ftype if arg == "self" else f"{arg}=self.{arg}"

    args = [argument(p.name) for p in newparams]
    args = ", ".join(args)

    code = PREAMBLE if preamble else ""
    # code = "# noinspection PyPep8Naming\n"

    code += f"@core.wrap_function({qname})\n"
    code += f"class {name}(model.Indicator):\n"

    if len(newparams) > 1:
        code += indent + f"def __init__{newsig}:\n"
        for p in newparams[1:]:
            name = p.name
            code += indent2 + f"self.{name} = {name}\n"
        code += "\n"

    code += indent + f"def __call__(self, {ftype}):\n"
    code += indent2 + f"return {qname}({args})\n"

    return code


print(make_indicator(core.EMA))

@core.wrap_function(core.EMA)
class EMA(model.Indicator):
    def __init__(self, period: int, *, adjust: bool = False, item: str = None):
        self.period = period
        self.adjust = adjust
        self.item = item

    def __call__(self, series):
        return core.EMA(series, period=self.period, adjust=self.adjust, item=self.item)



In [126]:
def test_indicator(func, name=None, verbose=False):
    code = make_indicator(func, name=name, preamble=True, verbose=verbose)
    print(code)
    return new_module("mintalib.functions", code)


test_indicator(core.EMA)


""" Mintalib Indicators """

# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from . import model

nan = float('nan')

@core.wrap_function(core.EMA)
class EMA(model.Indicator):
    def __init__(self, period: int, *, adjust: bool = False, item: str = None):
        self.period = period
        self.adjust = adjust
        self.item = item

    def __call__(self, series):
        return core.EMA(series, period=self.period, adjust=self.adjust, item=self.item)



<module 'mintalib.functions'>

In [127]:
def make_indicators(names=None):
    if names is None:
        names = core_functions()

    output = PREAMBLE

    buffer = pformat(names, width=75, compact=True, indent=4)
    buffer = buffer.replace("(", " ").replace(")", "")
    output += f"__all__ = [\n{buffer}\n]\n"

    output += "\n\n"

    for name in names:
        func = getattr(core, name)
        text = make_indicator(func)
        output += text + "\n\n"


    return output


code = make_indicators()
print(code[:256], "...")


""" Mintalib Indicators """

# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from . import model

nan = float('nan')

__all__ = [
    'AVGPRICE', 'TYPPRICE', 'WCLPRICE', 'MIDPRICE', 'PRICE', 'CROSSOVER',
    'CROSSUNDER ...


In [128]:
code = make_indicators()

indicators = new_module("mintalib.indicators", code)

dir(indicators)

['ADX',
 'ATR',
 'AVGPRICE',
 'BBANDS',
 'BOP',
 'CCI',
 'CMF',
 'CROSSOVER',
 'CROSSUNDER',
 'DEMA',
 'DIFF',
 'EMA',
 'EVAL',
 'EXP',
 'FLAG_ABOVE',
 'FLAG_BELOW',
 'FLAG_INVERT',
 'FLAG_UPDOWN',
 'FORECAST',
 'HMA',
 'KAMA',
 'KELTNER',
 'KER',
 'LATR',
 'LOG',
 'MA',
 'MACD',
 'MAD',
 'MAX',
 'MFI',
 'MIDPRICE',
 'MIN',
 'MINUSDI',
 'NATR',
 'PLUSDI',
 'PPO',
 'PRICE',
 'RMA',
 'ROC',
 'RSI',
 'RVALUE',
 'SAR',
 'SLOPE',
 'SMA',
 'STDEV',
 'STOCH',
 'STREAK',
 'SUM',
 'TEMA',
 'TRANGE',
 'TYPPRICE',
 'WCLPRICE',
 'WMA',
 '__all__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'core',
 'model',
 'nan']

In [129]:
assert callable(indicators.SMA)
assert callable(indicators.SMA(20))

res = indicators.SMA(20) @ prices

assert len(res) > 0


In [130]:
outfile = PKGDIR / "indicators.py"

print(f"Updating {outfile.name} ...")

outfile.write_text(code)

Updating indicators.py ...


14703

In [131]:
from mintalib import indicators

In [132]:
help(indicators)

Help on module mintalib.indicators in mintalib:

NAME
    mintalib.indicators - Mintalib Indicators

CLASSES
    mintalib.model.Indicator(builtins.object)
        ADX
        ATR
        AVGPRICE
        BBANDS
        BOP
        CCI
        CMF
        CROSSOVER
        CROSSUNDER
        DEMA
        DIFF
        EMA
        EVAL
        EXP
        FLAG_ABOVE
        FLAG_BELOW
        FLAG_INVERT
        FLAG_UPDOWN
        FORECAST
        HMA
        KAMA
        KELTNER
        KER
        LATR
        LOG
        MA
        MACD
        MAD
        MAX
        MFI
        MIDPRICE
        MIN
        MINUSDI
        NATR
        PLUSDI
        PPO
        PRICE
        RMA
        ROC
        RSI
        RVALUE
        SAR
        SLOPE
        SMA
        STDEV
        STOCH
        STREAK
        SUM
        TEMA
        TRANGE
        TYPPRICE
        WCLPRICE
        WMA
    
    class ADX(mintalib.model.Indicator)
     |  ADX(period: int = 14)
     |  
     |  Average Dir